# Turn raw EEG data into processed (epochs,eyes open, eyes closed, and noise) segments
#### Input: *.edf with recording data and the .csv containing timestamps for resting recording
####  Output: *-preprocessed-epo.fif, *-resting_EC-raw.fif, *resting_EO-raw.fif, *-noise_EO-raw.fif

In [1]:
import os
import mne
import pandas as pd
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
# sys.path.append('/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


In [2]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in data
data_dir = Path('../../Data/')
data_path = data_dir / 'EEG DATA/'
processed_data_path = data_dir / 'Processed Data/'
csv_path = data_dir / 'Eyes Timestamps/'
epo_path = processed_data_path 
save_path_resting = processed_data_path

save_paths=[save_path_resting,]
[os.makedirs(path, exist_ok=True) for path in save_paths];

[-2.5,0.0,2.5]


In [3]:
# viable subjects
sub_ids = utils.import_subs(data_dir,'sub_ids.txt')
# select only 64ch subs and turn into set
# sub_ids = sub_ids[7:]
print(sub_ids)

['007', '008', '009', '010', '013', '014', '015', '018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [4]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 44
Controls: 25
Total: 69


In [5]:
sub_ids = ['056']

In [6]:
for sub_id in sub_ids:
    # Preprocess continuous to eyes open, noise calibration, and eyes closed
    # raw, eyes_closed, noise_segment, eyes_open = preprocess.to_raw(data_path,sub_id,save_path=processed_data_path,csv_path=csv_path)
    raw = mne.io.read_raw_fif(epo_path / f'{sub_id}_preprocessed-raw.fif', preload=True)
    
    # stim_epochs = mne.read_epochs(epo_path / f'{sub_id}_preprocessed-epo.fif')
    stim_epochs, epo_times, stim_labels, pain_ratings = preprocess.to_epo(raw, sub_id, data_path, save_path=processed_data_path) 
    
    # check epochs for duration in seconds
    dur = stim_epochs.times[-1] - stim_epochs.times[0]
    print(f"{sub_id} duration: {dur}")
    if dur < 5.0:
        print(f"{sub_id} too short")
        continue

[-2.5,0.0,2.5]
056
Creating epochs WITH key presses

056
Removing repeated epochs from annotations...

Dropped 4 repeated epochs
056
finding the 60 pin-prick epochs...
<Epochs |  495 events (all good), -2.5 – 2.5 s, baseline 0 – 0 s, ~453.5 MB, data loaded,
 '1000001': 99
 '1000010': 99
 '1100001': 115
 '1100010': 115
 'Med Pain Back': 10
 'Med Pain Hand': 9
 'No Pain Back': 10
 'No Pain Hand': 10
 'Yes Pain Back': 10
 'Yes Pain Hand': 10
 and 5 more events ...>
[10, 11, 12, 13, 7, 4, 8, 5, 6, 3, 1, 2, 9]
['1000001', '1000010', '1100001', '1100010', 'Med Pain Back', 'Med Pain Hand', 'No Pain Back', 'No Pain Hand', 'Yes Pain Back', 'Yes Pain Hand', 'eyes closed', 'eyes opened', 'stop']
1100001
1000001
1000001
1000001
1100001
1000001
1000001
1100001
1100001
1000001
1000001
1100001
1100001
1000001
1100001
1100001
1000001
1100001
1000001
1000001
1100001
1000001
1100001
1000001
1000001
1100001
1000001
1100001
1100001
1100001
1100001
1000001
1000001
1100001
1000001
1100001
1000001
1100001
10

In [8]:
# NO Keypresses
sub_ids = ['045', 'C1.', 'C5.', 'C8.', 'C20', 'C21', 'C22']